<a href="https://colab.research.google.com/github/divyasrimandadi/ExcelR-Training/blob/main/RAG_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RAG stands for Retrieval-Augmented Generation

In [2]:
!pip install transformers

In [4]:
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
dataset = [
"To Kill a Mockingbird is a novel by Harper Lee, published in 1960, that explores issues of racial injustice and childhood innocence.",
"The Lord of the Rings is a high fantasy novel by J.R.R. Tolkien, published in three volumes between 1954 and 1955, that follows the quest of a hobbit to destroy the One Ring.",
"Pride and Prejudice is a romantic novel by Jane Austen, published in 1813, that explores the complexities of relationships and societal expectations in 19th-century England.",
"The Hunger Games is a dystopian novel by Suzanne Collins, published in 2008, that explores the dangers of totalitarianism and the effects of trauma on individuals.",
"1984 is a dystopian novel by George Orwell, published in 1949, that depicts a totalitarian future society where independent thought is discouraged and citizens are constantly surveilled.",
]
#initialize TF-IDF vectorizer
vectorizer=TfidfVectorizer()
tfidf_matrix=vectorizer.fit_transform(dataset)
query="Artificial Intelligence"
tfidf_vectors=tfidf_matrix.toarray()

#load pre-trained question answering model from Hugging Face
qa_pipeline=pipeline("question-answering",model="distilbert-base-cased-distilled-squad")
def retrive_and_answer(query,top_k=1):
  #converting query to tfidf vector
  query_vec=vectorizer.transform([query]).toarray()
  #computing cosine similarity btw query vector and all doc vectors
  similarities=cosine_similarity(query_vec,tfidf_vectors)
  print(similarities)
  #getting index of most similar passage
  most_similar_indices=np.argsort(similarities[0])[::-1][:top_k]
  print(most_similar_indices)
  #retriving most similar passages
  similar_passages=[dataset[i] for i in most_similar_indices]
  print(similar_passages)
  #combine passages to form context
  context=" ".join(similar_passages)
  #answer the question using qa model
  result=qa_pipeline(question=query,context=context)
  return result["answer"]
query="what does Kill a Mockingbird says"
answer=retrive_and_answer(query)
print(answer)

Device set to use cpu


[[0.3921383 0.        0.        0.        0.       ]]
[0]
['To Kill a Mockingbird is a novel by Harper Lee, published in 1960, that explores issues of racial injustice and childhood innocence.']
explores issues of racial injustice and childhood innocence
